## Scraping de texto sobre a lei rouanet

###  Daniel Silva Agostinho - 470435

#### Retirando dados do twitter utilizando a biblioteca Selenium

In [8]:
import csv
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.edge.service import Service

from selenium.webdriver import Edge

In [9]:
def transform_number(st_number):

    if st_number == "":
        return 0

    if st_number[-1] == "K":
        qtd = float(st_number[:-1]) * 1000
    elif st_number[-1] == "M":
        qtd = float(st_number[:-1]) * 1000000
    else:
        qtd = float(st_number.replace(',', ''))

    return int(qtd)

In [10]:
#Script

def get_data(tweet):
    #username
    try:
        #username
        username = tweet.find_element("xpath",'.//div[@data-testid="User-Name"]/div').text
        
        #arroba, ex: @daniel_agostin
        arroba = tweet.find_element("xpath",'.//span[contains(text(), "@")]').text
        
        #dia
        dia = tweet.find_element("xpath",'.//time').get_attribute('datetime')
        
        #texto do tweet
        conteudo = tweet.find_element("xpath",'.//div[@data-testid="tweetText"]').text
        
        #quantidade de comentarios
        comentarios = tweet.find_element("xpath",'.//div[@data-testid="reply"]').text
        comentarios = transform_number(comentarios)
        
        #quantidade de retweets
        ret = tweet.find_element("xpath",'.//div[@data-testid="retweet"]').text
        ret = transform_number(ret)
        
        #quantidade de likes
        likes = tweet.find_element("xpath",'.//div[@data-testid="like"]').text
        likes = transform_number(likes)
    except NoSuchElementException:
        return
    except StaleElementReferenceException:
        return
           
    tweet_id = arroba + dia
    
    return {'id':tweet_id, 
            "username":username, 
            "arroba":arroba, 
            "date_time":dia, 
            "texto":conteudo, 
            "coment_cont":comentarios, 
            "ret_cont":ret, 
            "likes_cont":likes}

In [6]:
service = Service("./msedgedriver.exe")
driver = Edge(service= service)
#acesssando login page do twitter
driver.get("https://twitter.com/i/flow/login")

In [7]:
barra_pesquisa = driver.find_element("xpath",'//input[@placeholder="Search Twitter"]')
barra_pesquisa.send_keys('lei rouanet until:2021-11-30 since:2020-07-30')
barra_pesquisa.send_keys(Keys.RETURN)

sleep(1)

#Aba de tweets mais recentes
mais_recentes = driver.find_element("xpath",'//div[@role="tablist"]/div[2]/a')
mais_recentes.click()

#lista de tweets
tweet_data = []

#lista de ids, evitar de pegar o mesmo tweet mais de uma vez
ids = set()

#quantidade minima de tweets a ser minerada
quantidade_min = 5

while (len(tweet_data) < quantidade_min):
    
    tweets = driver.find_elements("xpath",'//article[@data-testid="tweet"]')
    
    for tweet in tweets[-10:]:
        tweet_tuple = get_data(tweet)
        if tweet_tuple:
            if tweet_tuple['id'] not in ids:
                tweet_data.append(tweet_tuple)
                ids.add(tweet_tuple['id'])
            
    
    driver.execute_script('window.scrollBy(0, 25000);')
    sleep(1)
    
#Escrevendo csv com os dados
with open('twitter_video_1.csv', 'w', newline='', encoding = 'utf-8') as f:
    writer = csv.writer(f)
    for tweetD in tweet_data:
        writer.writerow(list(tweetD.values()))  